In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm


train_data = pd.read_parquet('train_data.pq')
sub = pd.read_csv('/content/sample_submission (2).csv')


train_data = train_data.loc[train_data['user_id'].isin(sub.user_id.unique())].copy()

vc = train_data.item_id.value_counts().reset_index()
vc.columns = ['item_id', 'count']
ones = vc.loc[vc['count'] == 1, 'item_id'].unique()
train_data = train_data.loc[~train_data.item_id.isin(ones)].copy()


last_day = int(train_data.date.max())
train_last_week = train_data.loc[train_data.date >= last_day - 10].copy()
use_items = train_last_week.groupby('user_id')['item_id'].apply(set).to_dict()


use_items_full = train_data.groupby('user_id')['item_id'].apply(set).to_dict()
top_last_week = train_data.loc[train_data.date == last_day, 'item_id'].value_counts()

top_last_week_list = top_last_week.index.tolist()


dict_top = {}
for x in tqdm(top_last_week_list[:120], desc='build dict_top'):
    users_ = train_data.loc[train_data.item_id == x, 'user_id'].unique()
    d = train_data.loc[train_data.user_id.isin(users_)]
    dg = d.loc[d.date == last_day, 'item_id']
    g = dg.value_counts().head(100).index.tolist()
    dict_top[x] = g

def exclude_items(cands, already):
    used = set(already)
    out = []
    for it in cands:
        if it not in used:
            out.append(int(it))
            used.add(int(it))
    return out


prediction = []
uids = sub.user_id.unique()

for user in tqdm(uids, desc='predict'):
    user_mini = list(use_items.get(user, set()))

    custom_top = [dict_top[x] for x in user_mini if x in dict_top]
    if len(custom_top) != 0:
        custom_top = custom_top[0]

    p = list(user_mini)
    if isinstance(custom_top, list):
        p += exclude_items(custom_top, list(user_mini))
    p += exclude_items(top_last_week_list, list(user_mini))

    prediction.append(p[:20])


rows = []
for uid, recs in zip(uids, prediction):
    for it in recs:
        rows.append([uid, it])

submission = pd.DataFrame(rows, columns=['user_id', 'item_id'])
submission.to_csv('submission_20.csv', index=False)
print('✓ submission_20.csv готов. Строк:', len(submission))

predict:  20%|█▉        | 57181/293230 [03:55<25:07, 156.63it/s]